In [1]:
from taker import Model
from datetime import datetime
import json
from os.path import exists

/workspace/SPAR/interp-la/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create helper function

In [2]:
import sys, os


class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## Load model

In [3]:
# model_name = "microsoft/phi-3-mini-4k-instruct"
model_name = "google/gemma-2-2b-it"
m = Model(model_name, dtype="int4")

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.22s/it]


Loaded model 'google/gemma-2-2b-it' with int4:
- Added 416 hooks across 26 layers


/workspace/SPAR/interp-la/.venv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [4]:
m.show_details()

 - n_layers : 26
 - d_model  : 2304
 - n_heads  : 8
 - d_head   : 256
 - d_mlp    : 9216


In [5]:
idlist = m.get_ids(".\n\n").squeeze().tolist()
print(idlist)
print(m.tokenizer.convert_ids_to_tokens(idlist))

idlist = m.get_ids("Sunday dinners.\n\n").squeeze().tolist()
print(idlist)
print(m.tokenizer.convert_ids_to_tokens(idlist))

idlist = m.get_ids("." * 16 * 5).squeeze().tolist()
print(idlist)
print(m.tokenizer.convert_ids_to_tokens(idlist))

idlist = m.get_ids("." * (16 * 5 + 1)).squeeze().tolist()
print(idlist)
print(m.tokenizer.convert_ids_to_tokens(idlist))

[2, 235265, 109]
['<bos>', '.', '\n\n']
[2, 20742, 90641, 235265, 109]
['<bos>', 'Sunday', '▁dinners', '.', '\n\n']
[2, 5519, 5519, 5519, 5519, 5519]
['<bos>', '................', '................', '................', '................', '................']
[2, 5519, 5519, 5519, 5519, 2779, 25984]
['<bos>', '................', '................', '................', '................', '........', '.........']


In [6]:
m.generate("Sunday dinners.\n\n", 25)

('Sunday dinners.\n\n',
 'A simple phrase that conjures images of a warm, inviting home, the comforting aroma of roasted chicken and vegetables, and the')

## Create prompts

In [7]:
prompt = """Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title."""


story = """
\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners.\n\n"""

prompt_original = prompt + story
print(prompt_original)

Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title.

 Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence

In [38]:
n_tokens_to_transfer = 1

idlist_original = m.get_ids(prompt_original).squeeze().tolist()
tokens_original = m.tokenizer.convert_ids_to_tokens(idlist_original)
n_tokens_original = len(tokens_original)

# # prompt is just string of dots.
# prompt_new = "." * 16 * n_tokens_to_transfer

prefix = ""
tokens_to_transfer = tokens_original[-n_tokens_to_transfer:]
string_to_transfer = m.tokenizer.convert_tokens_to_string(tokens_to_transfer)
prompt_new = prefix + string_to_transfer
print(f"{prompt_new=}")

idlist_new = m.get_ids(prompt_new).squeeze().tolist()
tokens_new = m.tokenizer.convert_ids_to_tokens(idlist_new)
n_tokens_new = len(tokens_new)

token_index_map = {
    n_tokens_original
    - n_tokens_to_transfer
    + i: n_tokens_new
    - n_tokens_to_transfer
    + i
    for i in range(n_tokens_to_transfer)
}

# do sense check
for index_original, index_new in token_index_map.items():
    print(repr(tokens_original[index_original]), repr(tokens_new[index_new]))

prompt_new='\n\n'
'\n\n' '\n\n'


## Transfer activations

In [39]:
# RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
for h in m.hooks.neuron_replace.values():
    h.reset()

print(m.hooks.neuron_replace)

{'layer_0_attn_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_0_mlp_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_1_attn_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_1_mlp_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_2_attn_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_2_mlp_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_3_attn_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_3_mlp_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_4_attn_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_4_mlp_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_5_attn_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_5_mlp_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_6_attn_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_6_mlp_pre_out': NeuronReplace(
  (param): ParameterDict()
), 'layer_7_attn_pre_out': NeuronReplace(
 

In [40]:
activations_original = m.get_midlayer_activations(prompt_original)

for original_index, new_index in token_index_map.items():
    for layer_type in ["mlp", "attn"]:
        # for layer_type in ["attn"]:
        for layer_number in range(26):
            hook = m.hooks.neuron_replace[f"layer_{layer_number}_{layer_type}_pre_out"]
            hook.add_token(
                new_index,
                activations_original[layer_type][0, layer_number, original_index],
            )

# for original_index, new_index in token_index_map.items():
#     for name, hook in m.hooks.neuron_replace.items():
#         # name is of the form "layer_{layer_number}_{layer_type}_pre_out"
#         _, layer_number, layer_type, _, _ = name.split("_")
#         layer_number = int(layer_number)
#         hook.add_token(new_index, activations_original[layer_type][0, layer_number, original_index])

print(m.hooks.neuron_replace)

{'layer_0_attn_pre_out': NeuronReplace(
  (param): ParameterDict(  (1): Parameter containing: [torch.cuda.HalfTensor of size 8x256 (cuda:0)])
), 'layer_0_mlp_pre_out': NeuronReplace(
  (param): ParameterDict(  (1): Parameter containing: [torch.cuda.HalfTensor of size 9216 (cuda:0)])
), 'layer_1_attn_pre_out': NeuronReplace(
  (param): ParameterDict(  (1): Parameter containing: [torch.cuda.HalfTensor of size 8x256 (cuda:0)])
), 'layer_1_mlp_pre_out': NeuronReplace(
  (param): ParameterDict(  (1): Parameter containing: [torch.cuda.HalfTensor of size 9216 (cuda:0)])
), 'layer_2_attn_pre_out': NeuronReplace(
  (param): ParameterDict(  (1): Parameter containing: [torch.cuda.HalfTensor of size 8x256 (cuda:0)])
), 'layer_2_mlp_pre_out': NeuronReplace(
  (param): ParameterDict(  (1): Parameter containing: [torch.cuda.HalfTensor of size 9216 (cuda:0)])
), 'layer_3_attn_pre_out': NeuronReplace(
  (param): ParameterDict(  (1): Parameter containing: [torch.cuda.HalfTensor of size 8x256 (cuda:0)])


## create outputs

In [41]:
# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
current_time = "2024-08-20_08-26-41"
filename = f"../results/{current_time}_agnes_multi_token_transfer_LA_tests.jsonl"

if not exists(filename):
    with open(filename, "w") as f:
        pass

In [42]:
max_new_tokens = 200
temperature = 0.01

# test on single output
output = m.generate(prompt_new, max_new_tokens, temperature=temperature)
print(output[1])

# with HiddenPrints():
#     for i in range(20):
#         output = m.generate(prompt_new, max_new_tokens, temperature=temperature)

#         data = {
#             "temperature": temperature,
#             "max_new_tokens": max_new_tokens,
#             "model": model_name,
#             "transplant_layers": (0, 25),
#             "transferred_token_num": n_tokens_to_transfer,
#             "orig_prompt": prompt_original,
#             "transplant_prompt": prompt_new,
#             "output": output[1],
#             "other_info": "try-different-transfer-layers-with-gemma",
#         }

#         with open(filename, "a") as file:
#             file.write(json.dumps(data) + "\n")

The recipe for this dish is a secret, but the ingredients are simple and readily available. The dish is best served hot and fresh.

**Ingredients:**

* 1 lb. boneless, skinless chicken breasts
* 1/2 cup chopped onion
* 1/2 cup chopped bell pepper
* 1/4 cup chicken broth
* 1/4 cup dry white wine
* 1 tablespoon olive oil
* Salt and pepper to taste
* 1/2 cup chopped fresh parsley

**Instructions:**

1. Heat olive oil in a large skillet over medium heat.
2. Add chicken breasts and cook until browned on both sides.
3. Add onion and bell pepper to the skillet and cook until softened.
4. Stir in chicken broth, white wine, salt, and pepper.
5. Bring the mixture to a boil, then reduce heat and simmer for 15 minutes.
6. Stir in parsley and serve hot.


In [43]:
for n_layers_transferred in range(1, 26, 2):
    # RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
    for h in m.hooks.neuron_replace.values():
        h.reset()

    activations_original = m.get_midlayer_activations(prompt_original)

    for original_index, new_index in token_index_map.items():
        for layer_type in ["mlp", "attn"]:
            # for layer_type in ["attn"]:
            for layer_number in range(n_layers_transferred):
                hook = m.hooks.neuron_replace[
                    f"layer_{layer_number}_{layer_type}_pre_out"
                ]
                hook.add_token(
                    new_index,
                    activations_original[layer_type][0, layer_number, original_index],
                )

    max_new_tokens = 100
    temperature = 0.01

    with HiddenPrints():
        for i in range(3):
            output = m.generate(prompt_new, max_new_tokens, temperature=temperature)

            data = {
                "temperature": temperature,
                "max_new_tokens": max_new_tokens,
                "model": model_name,
                "transplant_layers": (0, n_layers_transferred - 1),
                "transferred_token_num": n_tokens_to_transfer,
                "orig_prompt": prompt_original,
                "transplant_prompt": prompt_new,
                "other_info": f"transfer-first-{n_layers_transferred}-layers",
                "output": output[1],
            }

            with open(filename, "a") as file:
                file.write(json.dumps(data) + "\n")